In [4]:
from janome.tokenizer import Tokenizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
 
import pandas as pd

In [5]:
t = Tokenizer()
text="エネルギー"
 
tokens = t.tokenize(text)
for token in tokens:
    print(token)
 
print(type(token))
str(token)

エネルギー	名詞,一般,*,*,*,*,エネルギー,エネルギー,エネルギー
<class 'janome.tokenizer.Token'>


'エネルギー\t名詞,一般,*,*,*,*,エネルギー,エネルギー,エネルギー'

In [14]:
t = Tokenizer()
 
text="はじめに\n\n２０１１年３月の東日本大震災及び東京電力福島第一原子力発電所事故を受けて、政府は、２０１４年４月、２０３０年を念頭に、第４次エネルギー基本計画を策定し、原発依存度の低減、化石資源依存度の低減、再生可能エネルギーの拡大を打ち出した。"
 
tokens = t.tokenize(text)
 
df=pd.DataFrame()
for token in tokens:
    str_token=str(token).replace("\t",",")
    token_list=str_token.split(",")
    df= df.append(pd.Series(token_list),ignore_index=True)
 
df.columns = ["単語","品詞","品詞細分類1","品詞細分類2","品詞細分類3","活用型","活用形","原形","読み","発音"]
df.head()

,単語,品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用型,活用形,原形,読み,発音
0,はじめ,名詞,副詞可能,*,*,*,*,はじめ,ハジメ,ハジメ
1,に,助詞,格助詞,一般,*,*,*,に,ニ,ニ
2,\n\n,記号,空白,*,*,*,*,\n\n,*,*
3,２,名詞,数,*,*,*,*,２,ニ,ニ
4,０,名詞,数,*,*,*,*,０,ゼロ,ゼロ


In [18]:
#名詞（代名詞、非自立名詞をのぞく）を抽出
noun = df[df["品詞"]=="名詞"]
noun2=noun[(noun["品詞細分類1"] != "代名詞") & (noun["品詞細分類1"] != "非自立")]
noun2["単語"]

0       はじめ
3         ２
4         ０
5         １
6         １
7         年
8        ３月
10      東日本
11      大震災
13     東京電力
14       福島
16        一
17      原子力
18       発電
19        所
20       事故
25       政府
28        ２
29        ０
30        １
31        ４
32        年
33       ４月
35        ２
36        ０
37        ３
38        ０
39        年
41       念頭
45        ４
46        次
47    エネルギー
48       基本
49       計画
51       策定
54       原発
55       依存
56        度
58       低減
60       化石
61       資源
62       依存
63        度
65       低減
67       再生
68       可能
69    エネルギー
71       拡大
Name: 単語, dtype: object

In [17]:
#半角スペースで文字列連結
noun_df=" ".join(list(noun2["単語"])) 
noun_df

'はじめ ２ ０ １ １ 年 ３月 東日本 大震災 東京電力 福島 一 原子力 発電 所 事故 政府 ２ ０ １ ４ 年 ４月 ２ ０ ３ ０ 年 念頭 ４ 次 エネルギー 基本 計画 策定 原発 依存 度 低減 化石 資源 依存 度 低減 再生 可能 エネルギー 拡大'

In [69]:
import sqlite3
conn = sqlite3.connect("../fishing_result.db")
sql = '''
select
    *
from
    fishing_comments c
'''
df=pd.read_sql_query(sql, conn)
#comments=df[0:2]
comments=df
#comments

In [73]:
t = Tokenizer()
words=pd.DataFrame()

for comment in comments.to_dict('records'):
    # コメントを形態素解析
    print(comment['Comment'])
    if not comment['Comment']:
        continue
    tokens = t.tokenize(comment['Comment'])
    #print(len(list(tokens)))
    #if not tokens:
    #    continue
    df_token=pd.DataFrame()
    for token in tokens:
        str_token=str(token).replace("\t",",")
        token_list=str_token.split(",")
        #print(token_list)
        df_token = df_token.append(pd.Series(token_list),ignore_index=True)
    print(df_token.size)
    df_token = df_token.rename(columns={0:"単語",1:"品詞",2:"品詞細分類1"})

    #名詞（代名詞、非自立名詞をのぞく）を抽出
    noun = df_token[df_token["品詞"]=="名詞"]
    noun2=noun[(noun["品詞細分類1"] != "代名詞") & (noun["品詞細分類1"] != "非自立")]
    noun3 = pd.DataFrame(noun2["単語"])
    noun3['Date'] = comment['Date']
    noun3['Point'] = comment['Point']
    words = words.append(noun3, ignore_index=True)

サビキ釣りではイワシ・アジが釣れ、その他ではクロダイ・コノシロが好調でした。夕時からはカサゴ・メバルの良型が釣れていました。
380
サビキ釣りで限られた場所ですがイワシ・アジが釣れ、ご家族連れにも楽しんで頂けました。夕方からはメバルが釣れています。また、クロダイを専門に狙われている方は好調で、良型が上がっています。
590
サビキ釣りはコノシロが1日各所で釣れ好調です。イワシは午前中が中心で外側先端付近で釣れました。アジは夕方4時30分過ぎから外側の中央から先にかけて回遊があり、足元でも釣れていました。
560
サビキ釣りは、朝と夕方を中心に釣果がありました。特に夕マヅメで数が伸び、朝は内側、夕方は外側、共に中央から先が良かったようです。
420
サビキ釣りのイワシは、再び増加傾向になりました。そのイワシを追って来たヒラメが、泳がせ釣りで上がりました。コノシロも、桟橋中央から先端で釣れています。
430
サビキ釣りのイワシ・コノシロが好調に釣れ続けていて、お客様みなさまにお楽しみいただきました。アジは両先端付近を中心に夕方に投げサビキで釣果がでました。またイワシを追いかけてスズキ・ヒラメ等が入って来ていて、イワシの泳がせ釣りで釣れました。その他、夕方の閉場前にメバルも釣れています。
830
サビキ釣りのイワシ・コノシロが好調に釣れ続けて、ファミリーのお客様にお楽しみ頂いております。コブダイの特大サイズ67cm(8kg)が牡蠣のエサで上がりました。サビキ釣りで釣れたイワシにはヒラメが食いつきました。夕方、釣り方制限の解除後に遠投サビキ釣りでアジが釣れだしました。
780
昨日に引き続きサビキの釣果が良く、イワシ・コノシロが数釣りされています。アジは場所によって差があるようですが、潮濁りが入れば広い範囲で期待出来そうな気配です。メジナ・クロダイも専門に狙って型を見ており、朝方にはルアーシーバスもキャッチされています。
730
朝方にルアー釣りでシーバスが上がりました。サビキ釣りでは、一日を通してシコイワシ・コノシロが釣れ続け、小さなお子様にも楽しんで頂けました。また、クロダイを専門に狙われている方が、良型のクロダイをおもちでした。
600
ここ数日好調のシコイワシ・コノシロが本日もたくさん釣れました。アジは限られた場所で釣れ、20匹以上釣られたお客様も数人いました。そ

1270
朝からアジ・サッパが好調に釣れ続き、特に中央より先でサビキ釣りが賑わってました。多い方でアジ(20cmUP)40匹、50匹の方もいらっしゃいました。ルアーシーバスが思ったより数は出ていませんが、泳がせではバラシも含め魚影は充分感じられます。
700
サビキ釣りで朝からアジ・イワシ・サッパが釣れ、イワシ・サッパは数をお持ちの方が複数いらっしゃいました。ルアーの釣果は、朝イチにスズキクラスが2匹、イナダが1匹上がっていました。
470
サビキ釣りのアジ・イワシは好調をキープしており、本日も数を伸ばしました。釣り方制限でルアーの釣果が振るわない中、泳がせ釣りで年間大物賞更新のブリが釣れ、スズキ・カマスも釣れました。14日からの平日、釣り方制限がかからなければルアーの釣果に期待大です。
750
1日を通してサビキ釣りのイワシ・アジが好調に釣れ続けて、ファミリーのお客様に喜ばれて多くの笑顔を頂きました。アジの泳がせ釣りで年間大物賞候補更新のキジハタが釣れました。
480
本日もルアーシーバスが好調でしたがほとんどが午前中の釣果でした。朝の開場後にはルアーでの年間大物賞候補となる90cmのスズキも上がりました。サビキ釣りでは、場所によっては1日中ポツポツとアジが釣れ続き数が伸びました。午後3時30分頃からは足元で釣れるほどアジの回遊がありました。
840
本日もルアーシーバスが好調で数が伸びました。最大のスズキはアジの泳がせで釣れました。サビキ釣りは、内側のピンスポットの投げサビキで一日を通じ釣れ続け、1人で70匹釣っていました。その他のアジは夕方に回遊があり数が伸びました。クロダイは、カゴ釣りでの釣果です。
790
シーバス好調でスズキクラスが4匹上がりました。サビキ釣りは、午前中に狭い範囲でアジが釣れ、夕方にアジの回遊があり釣果が伸び、イワシは昼頃から内側で釣果が出ていました。その他ではイワシの泳がせ釣りでイナダが釣れていました。
680
先週から続くイワシの釣果のお蔭か、フィッシュイーターの活性も高まっているようです。タチウオ・イナダ・フッコとルアーでの釣果が上がっています。人気のアジも朝夕はもちろん、日中もポツポツながら釣れ続け、50匹オーバーを釣られた強者もいらっしゃいました。場所は限定になります。
720
一日を通してイワシが釣れ続けて、ファミリーのお客様

730
サビキ釣りのアジは大半が豆アジですが、チョイ投げのシロギスの釣果は好調です。ルアー釣りでは、朝方にシーバスがヒットしています。活きエサ釣りでのマゴチも、大型が狙えます。
500
サビキ釣りのアジは、30cm後半の大型が釣れました。活きエサのマゴチ仕掛けも大型が狙えます。
270
サビキ釣りのアジは、夕マヅメに中型が回ってきました。投げ釣りのシロギスは、好調を維持しています。
310
日中は釣果も苦戦しましたが、夕方の下げ潮では桟橋内側で22cm前後のアジの回遊がありました。クロダイも活性が上がったようです。
410
サビキ釣りのアジは豆アジ8割、中型2割です。投げ釣りのイシモチは良いポイントを掴んだ方が数を揃えています。
340
サビキ釣りでイワシ・豆アジ、チョイ投げでシロギス・イシモチ、大物狙いではハゼの泳がせでマゴチが好調です。
280
サビキ釣りでは、アジの釣果は殆どが豆アジでしたが、25cm前後の良型も多少交じっています。イワシは本日も好調で数が出ています。投げ釣りのシロギスも良く1人で10匹以上お持ちの方もいらっしゃいました。マゴチも安定して釣れています。
740
マゴチが年間大物賞候補更新の60cmを頭に数が揃いました。サビキ釣りは桟橋全体で朝からイワシが好調で、ファミリーのお客様に喜ばれました。アジは夕マヅメに外側先端で25cmクラスが回ってきました。数を釣られる方は15m程遠投して、タナは底を狙っています。ルアー釣りでイワシを追って来たタチウオが、朝夕のマヅメ時に指4本クラスが釣れています。食べて美味しい魚のコショウダイが桟橋内側で上がっています。投げ釣りのイシモチは旨く群れを探した方が数を釣られています。
1430
サビキ釣りは、全体的にイワシが回っていますが、アジ・サバは少し交じっている状況です。アジは、殆んどが豆アジの釣果です。ルアーでは、朝方にタチウオ・シーバスが顏を見せています。エサ釣りの釣果は、クロダイ・キビレ・メジナが出ています。
670
ルアー75cmのスズキが夕方に上がりました。その他マゴチ、泳がせ釣りでフッコも上がっていました。サビキ釣りは殆どが豆アジの釣果です。夕方閉館間際に20cm前後の物が釣れ出しましたが数は伸びませんでした。
600
釣果はイワシの泳がせで62cmのスズキが出ています。サビキ釣りは残念ながら昨日に続

500
ルアーシーバスは昨日同様に不調でしたが、夕方に多くのバイトがあったようです。また、サビキ釣りのアジも閉場の一時間前(16時頃)に時合があり、多い方で20匹釣られていました。好調なウミタナゴ・根魚のカサゴ・ソイの他に、4日ぶりのクロダイも姿を見せています。
800
月間大物賞の2.5kgのタコが上がりました。ルアーシーバスは朝方フッコサイズが2匹出ました。ウミタナゴの26cmが釣れました。2月の月間大物賞に27cm以上が設定されましたので是非狙って下さい。
580
好調を維持したルアーシーバスが数を伸ばしており、またサビキ釣りでは珍しくタコも釣れておりました。月間大物賞となるタコ(1.8kg以上)とルアーシーバス(75cm以上)が受賞されております。
550
早朝のシーバスは時合は短いものの活性が高く、また日中はイソメエサ、サビキ仕掛けで釣れていました。アジは多い方で8匹釣られ、朝に比べ夕方に数を伸ばされたようです。その他ではウミタナゴなど冬の魚が釣れていました。
680
夕マヅメ16:00以降に内側先端にアジが回って来て、遠投サビキ釣りで釣れていました。サビキ釣りには、ウミタナゴも好調でした。2月の月間大物賞に27cm以上が設定されましたので是非狙って下さい。月間大物賞にあと一歩の、73cmのシーバスがバイブレーションで上がっています。投げ釣りで、32cmのイシモチが上がりました。
950
アジは15:30頃から内側中央で釣れていました。サビキ釣りは日中はウミタナゴメインになっています。午前中にルアーでシーバス大型が3匹、投げ釣りでカレイが2匹。夕方にメバルの良型が出ています。
600
お客様が少なく釣果もあまり伸びませんでした。そんな中、マダコ2.1kg大物賞が出ています。ルアーのシーバスも67cm、48cmが出ています。毎年大黒のシーバスは一月下旬に良型が出ていますので、これからに期待したいところです。アジは夕マヅメに遠投して釣れたそうです。
820
冬のこの時期は早朝のシーバスは好調なものの、サビキ釣り、投げ釣り、ヘチ釣りなど厳しい釣果が続いています。タコの月間大物賞が2匹上がっています。
430
この時期特有の釣果が続いている日々ですが、今日も魚種、魚数共に低い釣果となっております。厳しい中でも早朝からランカークラスを筆頭にルアーシーバスが4匹上がり

1070
朝と夕方にサバが回っています。アジも夕方に回り、豆～レギュラーサイズが釣れています。今日は大アジは少なくなっています。ここの所アイゴが釣れています。ヒレに毒がありますのでくれぐれも素手ではお持ちにならないようご注意ください。
690
穏やかな釣り日和となりました。40cm前後の大アジやサバが釣れていますが仕掛けや場所で釣果に差が出ている様です。カサゴの大物賞29cmも出ています。
490
サビキ釣りのアジ・サバは若干数を落としましたが、閉場間際に数釣りをされる光景がありました。カワハギは数こそ出ていませんが、月間大物賞と年間大物賞候補を同時に受賞されるサイズが釣れております。
600
朝から大アジが釣れて、年間大物賞候補も出ました。大アジを狙うなら、今がチャンスです。サバ・サワラ・イナダが釣れています。タコも、まだ狙えます。
460
サビキでサイズの良いアジ・サバが釣れ、青物ではシーバス・イナダ・ワカシが釣れました。その他はメジナ・カサゴ・カワハギの月間大物賞が釣れました。
360
前回同様に初日の釣果を期待しておりましたが、青潮による影響なのかサビキ釣り、投げ釣りの釣果が思わしくありませんでした。カサゴ・メジナ等、障害物に付いている魚は安定しており、また青物ではイナダ・サワラ・ショゴ等の姿が見られます。
680
台風の影響で波が荒く厳しい状況でしたが、皆さん頑張って釣りをしていらしゃいました。朝からアジが37cmを筆頭に釣れていました。またへチ釣りで40cmを筆頭にクロダイが釣れ、99cmのタチウオも釣れました。
640
早朝の時間帯に良型のアジ・サワラが上がり、釣り場に活気が見られました。日中はカサゴ・イシモチ等数釣りをされるお客さまもいらっしゃいました。夕マヅメ前からサビキのアジやサバが釣れだして、良型30cmオーバーも多数お持ちでした。ルアーでは数は出ませんでしたが、イナダ・タチウオがキャッチされています。
910
アジは昨日と同程度の数が出ており、今日は40cm近い大型のものも数匹見られました。その他の目立ったところでは、エサ釣りで67cmのスズキが釣れ、カサゴが引き続き数を伸ばしており、魚種を絞れば楽しめる状況かと思います。
680
引き続き朝夕のアジが出ていますが、数日前に比べ少しずつ数が減っています。その他まだ数が少なめですが良型のカワハギ

...

沖桟橋ではアジ・ウミタナゴ・カレイ・コノシロ・ハゼ等。新護岸ではアナゴ・シロギス・タコ・フグ等。旧護岸ではイイダコ・セイゴ等。渡桟橋ではカサゴ・クロダイ。全釣場では対象魚種無し。昨日に引き続き良型のコノシロ・ウミタナゴが釣れました。アジは食い渋りながらもポツポツと釣れていました。2月の月間大物賞となる51cmのクロダイが出ましたが、全体としての釣果は厳しい1日となってしまいました。
1060
沖桟橋ではアジ・イシモチ・ウミタナゴ・クロダイ・コノシロ・サバ・メゴチ等。新護岸ではアイナメ・アナゴ・カレイ・タコ・フグ等。旧護岸ではイイダコ・シロギス・セイゴ・ハゼ等。渡桟橋ではカサゴ等。日中はお天気にも恵まれ釣り日和の1日となりました。釣果はサビキ仕掛けでウミタナゴ・コノシロが目立ちました。投げ仕掛けではアナゴ・カレイ・シロギス・イシモチ等が釣れましたが、全体としては釣果は渋い状況でした。
990
沖桟橋ではウミタナゴ・コノシロ・シロギス・ハゼ等。新護岸ではアナゴ・クロダイ・フグ等。
180
沖桟橋ではアジ・イイダコ・ウミタナゴ・カサゴ・コノシロ・フッコ・セイゴ等。新護岸ではアナゴ・カレイ・フグ等。旧護岸ではカサゴ・シロギス・ハゼ・メバル等。終日、釣場には寒風が吹き、我慢の釣りとなりました。ウミタナゴは良く釣れていましたが、全体的には渋い状況でした。
680
沖桟橋ではアジ・ウミタナゴ・カサゴ・コノシロ・シロギス・ハゼ等。新護岸ではアジ・イシモチ・カレイ・フッコ・ハゼ・フグ・メバル等。旧護岸ではアイナメ・ウミタナゴ・カサゴ・フグ・メバル等。午前中は穏やかな天候ながら中々釣果に結び付かず、午後からは南西の強風の為、沖桟橋並びに渡桟橋の閉鎖で更に釣果に恵まれずで、厳しい1日となってしまいました。
810
沖桟橋ではアジ・イシモチ・ウミタナゴ・サバ等。旧護岸ではタコ・イカ等。朝からの冷たい雨でご来場のお客様も少なく、その人数に比例して釣果も少なく終了しました。
430
沖桟橋ではアジ・ウミタナゴ・コノシロ・スズキ・セイゴ等。新護岸ではイシモチ・ウミタナゴ・フッコ等。旧護岸ではカレイ・シロギス等。大物賞スズキが釣れ、またアジが少し上向いていましたが、全体としては釣果は厳しいものとなりました。
520
新護岸ではスズキ等。旧護岸ではカレイ・シロギス・ハゼ等。渡桟橋ではクロダイ・フッコ

1340
沖桟橋ではアジ・イシモチ・イワシ・カレイ・クロダイ・コノシロ・サバ・シロギス・フッコ・セイゴ・マダイ等。渡桟橋ではアジ・イワシ・クロダイ・フッコ・セイゴ等。新護岸ではアナゴ・イシモチ・カワハギ・クロダイ・サバ・シロギス・フッコ・タコ等。旧護岸ではアイナメ・アオリイカ・イイダコ・カワハギ・スミイカ・タコ等。全釣場ではウミタナゴ・カサゴ等。久々に15時過ぎからアジの時合があったようです。
740
沖桟橋ではイシモチ・イワシ・サッパ・サバ・サワラ・フッコ・タコ・タチウオ等。渡桟橋ではイワシ・ウミタナゴ等。新護岸ではアナゴ・ウミタナゴ・フッコ・タチウオ・フグ等。旧護岸ではイイダコ・ウミタナゴ・カサゴ・カワハギ・ソイ・ハゼ・メバル等。全釣場ではアジ。終日、雨風に見舞われ釣り辛い1日でしたが、日中を通して曇天のせいか良型のアジ・サバがやや多く釣り上がった感じがしました。
730
沖桟橋ではアジ・イイダコ・イシモチ・イワシ・ウミタナゴ・クロダイ・コノシロ・サゴシ・サッパ・サバ・サワラ・シロギス・セイゴ・タコ・フグ・マダイ等。渡桟橋ではアジ・イワシ・コノシロ・サッパ等。新護岸ではウミタナゴ・カサゴ・コノシロ・サワラ・シャコ・スズキ・フグ等。旧護岸ではアイナメ・イイダコ・イワシ・ウミタナゴ・カサゴ・カレイ・カワハギ・タコ・ハゼ・フグ・マダイ。夕方のマヅメにアジがパタパタっと上がり出しましたが、型は小振りなサイズが多かった様でした。また、本日は2匹もスタッフビックリ賞のマダイ(60～67㎝)が上がりました。ルアーも午前中は調子が良かった模様です。
870
沖桟橋ではアジ・イワシ・カレイ・コノシロ・サッパ・サバ・フッコ等。新護岸ではアジ・イシモチ・ウミタナゴ・カレイ・スズキ・タチウオ等。旧護岸ではイイダコ・ウミタナゴ・カワハギ・サッパ・タコ等。渡桟橋ではアジ・イワシ・サバ・フッコ・ハゼ等。全釣場ではカサゴ・フグ。日中に何度もナブラが出ており、多くの方がルアーをキャストしておりましたがあまり釣果には繋がりませんでした。サビキもイワシが多少釣れていましたが、アジも含め全体的に渋い1日でした。
1040
沖桟橋ではアジ・アナゴ・イイダコ・イワシ・カレイ・クロダイ・コノシロ・サッパ・サバ・シロギス・スズキ・タチウオ・ハゼ・ボラ等。渡桟橋ではアジ・セイゴ等。新護岸ではアジ・イシモチ・ウミ

830
沖桟橋ではイシモチ・カレイ・コノシロ・サッパ・シロギス・フッコ等。新護岸ではイシモチ・カワハギ・クロダイ・コノシロ・シロギス等。旧護岸ではカレイ・カワハギ・サッパ・シロギス・メバル等。渡桟橋ではウミタナゴ・クロダイ・サッパ・フッコ・メバル等。全釣場ではアジ・カサゴ・ハゼ・フグ。朝から日中にかけては釣果が渋いながらも、釣り大会はお陰様で無事終了致しました。全体的な釣果はカワハギが良かった他、一部では日没後のアジが良かったようです。また、シロギスも数が出ていますが、ポイントが限られており狙うのは難易度が高そうです。
1240
沖桟橋ではアジ・サッパ・フッコ等。新護岸ではアジ・イシモチ・スズキ等。旧護岸ではカワハギ・メジナ・メバル等。渡桟橋ではアジ・クロダイ・サッパ等。全釣場ではカサゴ。
430
沖桟橋ではアナゴ・コノシロ・サッパ・スズキ・タチウオ等。新護岸ではサッパ・サワラ・シロギス・フッコ・タチウオ等。旧護岸ではアナゴ・カワハギ・シロギス・スズキ・メバル等。渡桟橋ではウミタナゴ・サッパ・フッコ・ベラ・メバル等。全釣場ではアジ・カサゴ・クロダイ・フグ。サッパ好調ですが、他に泳がせのアナゴ・イナダ・サワラ・スズキ等が目立ちました。
710
沖桟橋ではコノシロ・サッパ・サバ・スズキ・タチウオ等。新護岸ではイシモチ・サバ・サワラ・フッコ・タチウオ等。旧護岸ではウミタナゴ・カサゴ・カワハギ・フグ等。渡桟橋ではカサゴ・カワハギ・クロダイ・サッパ・セイゴ等。全釣場ではアジ。アジ・サッパの数が出ましたが、型の良いものは少なかったです。
610
沖桟橋ではアジ・サバ・サワラ・イワシ・タチウオ・カサゴ・シロギス・フッコ等。渡桟橋ではアジ・サバ・カサゴ・カワハギ・フッコ・メジナ・メバル等。新護岸ではアジ・サバ・イシモチ・カサゴ・クロダイ・シロギス・サワラ・タチウオ等。旧護岸ではアジ・サバ・カレイ・カワハギ・クロダイ・シロギス・アイナメ・アオリイカ等。
680
沖桟橋ではイワシ・クロダイ・コノシロ・サバ・サワラ・サゴシ・スズキ・フッコ・セイゴ・タチウオ等。渡桟橋ではイワシ・カサゴ・カワハギ・クロダイ・コノシロ・サバ・シマダイ・セイゴ・メジナ・メバル等。新護岸ではカサゴ・カワハギ・サゴシ・フッコ・シロギス・セイゴ等。旧護岸ではイワシ・カサゴ・カレイ・カワハギ・サバ・コノシロ・メジナ等。

1190
沖桟橋ではアジ・フッコ・アジ・イワシ・ウミタナゴ・サバ等。新護岸ではカサゴ・サバ・イワシ・フッコ等。旧護岸ではカサゴ・シマダイ・フッコ・メジナ等。渡桟橋ではアジ・サバ・シマダイ・カサゴ・メジナ等。終日に渡って雨が降ったり止んだりで非常に釣り辛いコンディションでした。サバは大型の物は41cmで、小型の物も20cm前後はあるようでした。イワシは時折、ウルメイワシも交じったようです。
880
沖桟橋ではアジ・ウミタナゴ・クロダイ・シコイワシ・シロギス・フッコ・セイゴ等。新護岸ではウミタナゴ・カサゴ・シロギス・スズキ・フッコ・セイゴ・タコ・メジナ等。旧護岸ではカサゴ・クロダイ・シロギス・メジナ等。渡桟橋ではアジ・クロダイ・シコイワシ・メジナ等。全釣場ではサバ。終日風が強く釣り辛いコンディションでした。サバは小型中心でイワシはウルメイワシも交じりました。メジナの大物賞が出ました。
810
沖桟橋ではアジ・イワシ・コノシロ・フッコ等。新護岸ではアジ・イワシ・シロギス・フッコ等。旧護岸ではアジ・イワシ・スズキ・ベラ等。渡桟橋ではクロダイ・シマダイ・メジナ等。全釣場ではウミタナゴ・サバ。強風で17時前に桟橋閉鎖となりましたが、その後も護岸でサバが釣れて数が伸びました。ただ、型の良い物は未だ数%程度といったところです。
960
沖桟橋ではアジ・イシモチ・クロダイ・シロギス・スズキ等。新護岸ではアジ・イシモチ・カサゴ・タコ・フグ・メジナ等。旧護岸ではイイダコ・ウミタナゴ・タコ・ハゼ等。渡桟橋ではアジ・カサゴ・イワシ・シマダイ・セイゴ等。午後から風が強くなってしまい、夕マヅメの釣果も振るわず全体的に渋めで終了しました。
730
沖桟橋ではアジ・シロギス・フッコ・セイゴ・イシモチ・ウミタナゴ・カサゴ・カレイ・コノシロ・イワシ等。渡桟橋ではアジ・ウミタナゴ・シマダイ・メジナ・セイゴ等。新護岸ではシロギス・スズキ・フッコ・タコ・イシモチ等。旧護岸ではタコ・ウミタナゴ・カサゴ等。サバは大小含めて全域で釣れていました。アジのスタートは早く午後4時位から、釣果の個人差が大きい結果となりました。
720
沖桟橋ではアジ・イシモチ・ウミタナゴ・クロダイ・コノシロ・シロギス・フッコ・セイゴ・タコ・メゴチ等。渡桟橋ではイシモチ・クロダイ・シマダイ・セイゴ・メバル等。新護岸ではアナゴ・イシモチ・シロ

690
沖桟橋ではアイナメ・アジ・イイダコ・イシモチ・カレイ・コノシロ・シマダイ・シリヤケイカ・シロギス・フッコ・セイゴ等。渡桟橋ではアジ・スズキ・メジナ等。新護岸ではイシモチ・カレイ・フッコ等。旧護岸ではカレイ・シロギス・ハゼ等。全釣場ではアナゴ・ウミタナゴ・カサゴ・クロダイ・メバル。期待してたアジの夕マヅメの釣果が振るわず、トータル305匹でした。良型のメジナ・シマダイも沖桟橋で上がり始めています。
810
沖桟橋ではアジ・アナゴ・イシモチ・カレイ・クロダイ・コノシロ・サバ・サヨリ・シマダイ・フッコ・セイゴ・ベラ等。渡桟橋ではアジ・クロダイ・ベラ等。新護岸ではアナゴ・イシモチ・カレイ・フッコ・セイゴ・ベラ等。旧護岸ではカレイ等。全釣場ではウミタナゴ・カサゴ・シロギス・メバル。朝一、午後一、夕マヅメと沖でアジの時合が有り、多い方で20～30匹と持ってました。カレイの大物賞も旧護岸で出ております。
920
沖桟橋ではアジ・アナゴ・セイゴ・イシモチ等。渡桟橋ではアジ・メジナ等。新護岸ではアナゴ・セイゴ・イシモチ・クロダイ等。旧護岸ではカレイ・メジナ・メバル等。全釣場ではウミタナゴ・カサゴ等。メバル大物賞28cmが出ました。旧護岸のメバルだけ好調でした。
730
沖桟橋ではアジ・イシモチ・カレイ・クロダイ・コノシロ・シマダイ・シロギス・メジナ等。渡桟橋ではアイナメ・アナゴ・シコイワシ・セイゴ・メジナ等。新護岸ではアジ・アナゴ・イシモチ・シコイワシ・シロギス・メジナ等。旧護岸ではアナゴ・カレイ・シコイワシ・シロギス・セイゴ・メジナ等。全釣場ではウミタナゴ・カサゴ等。アジは盛り上がりには欠けましたが終日ポツリポツリ最終329匹となりました。
550
沖桟橋ではアジ・イシモチ・ウミタナゴ・カレイ・クロダイ・コノシロ・サバ・サヨリ・シコイワシ・シロギス・フッコ・セイゴ・メバル等。渡桟橋ではアジ・カサゴ等。新護岸ではアジ・アナゴ・イシモチ・ウミタナゴ・クロダイ等。旧護岸ではアイナメ・アナゴ・ウミタナゴ・クロダイ・キビレ等。アジは沖桟橋で朝一とその後は16時位から渡桟橋と沖桟橋で好調な時間帯がありました。
590


In [74]:
words.describe()

,単語,Date,Point
count,50758,50758,50758
unique,3608,997,3
top,等,2019-05-23,Honmoku
freq,3160,124,18394


In [75]:
wc = words.groupby(['単語']).size()
wc.head()

単語
!      4
!?     1
%      2
&      2
(     88
dtype: int64

In [77]:
wc2 = wc.reset_index(name='count').sort_values(['count'], ascending=False)

In [78]:
words.to_csv('words.csv')

In [79]:
wc2.to_csv('word_count.csv')